<a href="https://colab.research.google.com/github/Agamjeet/Synthetic_voice_detection/blob/master/Spacy_NERipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import spacy

In [ ]:
!python -m spacy download en_core_web_lg

In [4]:
nlp=spacy.load("en_core_web_lg")
nlp


In [6]:
#once we have spacy language model we can pass any text into it and it will create a doc object
doc=nlp("Donald Trump was President of USA")

In [7]:
doc

Donald Trump was President of USA

In [8]:
type(doc)

spacy.tokens.doc.Doc

In [12]:
doc.ents#list of all entities in doc

(Donald Trump, USA)

In [15]:
doc.ents[0],type(doc.ents[0])
#each entity inside of doc.ent is of type span
#to train spacy model -
# 1 take each train example
# 2 create a doc onject
# 3 add entities in example as a list of span objtes
# 4 train spacy model
# 5 use visualization

(Donald Trump, spacy.tokens.span.Span)

In [17]:
from spacy import displacy
displacy.render(doc,style="ent",jupyter=True)

In [18]:
#use some data from kaggle
#https://www.kaggle.com/datasets/finalepoch/medical-ner?resource=download
import json
with open('/content/Corona2.json','r') as f:
  data=json.load(f)

In [ ]:
data['examples'][0]
#each example is a dict

In [21]:
data['examples'][0].keys()

dict_keys(['id', 'content', 'metadata', 'annotations', 'classifications'])

In [53]:
data['examples'][0]['content']

"While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]"

In [24]:
data['examples'][0]['annotations'][0]
#we are interested in tag_name and start and end

{'id': '0825a1bf-6a6e-4fa2-be77-8d104701eaed',
 'tag_id': 'c06bd022-6ded-44a5-8d90-f17685bb85a1',
 'end': 371,
 'start': 360,
 'example_id': '18c2f619-f102-452f-ab81-d26f7e283ffe',
 'tag_name': 'Medicine',
 'value': 'Diosmectite',
 'correct': None,
 'human_annotations': [{'timestamp': '2020-03-21T00:24:32.098000Z',
   'annotator_id': 1,
   'tagged_token_id': '0825a1bf-6a6e-4fa2-be77-8d104701eaed',
   'name': 'Ashpat123',
   'reason': 'exploration'}],
 'model_annotations': []}

In [26]:
training_data =[]
for example in data['examples']:
  temp_dict={}
  temp_dict['text']=example['content']
  temp_dict['entities']=[]
  for annotations in example['annotations']:
    start=annotations['start']
    end=annotations['end']
    label=annotations['tag_name'].upper()
    temp_dict['entities'].append((start,end,label))
  training_data.append(temp_dict)
print(training_data[0])

{'text': "While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]", 'entities': [(360, 371, 'MEDICINE'), (383, 408, 'MEDICINE'), (104, 112, 'MEDICALCONDITION'), (679,

In [27]:
training_data[0]['entities']

[(360, 371, 'MEDICINE'),
 (383, 408, 'MEDICINE'),
 (104, 112, 'MEDICALCONDITION'),
 (679, 689, 'MEDICINE'),
 (6, 23, 'MEDICINE'),
 (25, 37, 'MEDICINE'),
 (461, 470, 'MEDICALCONDITION'),
 (577, 589, 'MEDICINE'),
 (853, 865, 'MEDICALCONDITION'),
 (188, 198, 'MEDICINE'),
 (754, 762, 'MEDICALCONDITION'),
 (870, 880, 'MEDICALCONDITION'),
 (823, 833, 'MEDICINE'),
 (852, 853, 'MEDICALCONDITION'),
 (461, 469, 'MEDICALCONDITION'),
 (535, 543, 'MEDICALCONDITION'),
 (692, 704, 'MEDICINE'),
 (563, 571, 'MEDICALCONDITION')]

In [29]:
from spacy.tokens import DocBin
#object that stores doc
from tqdm import tqdm
nlp=spacy.blank("en") #Blank spacy model
doc_bin=DocBin()

In [30]:
from spacy.util import filter_spans
for training_example in tqdm(training_data):
  text=training_example['text']
  labels=training_example['entities']
  doc=nlp.make_doc(text)
  ents=[]
  for start,end,lable in labels:
    span=doc.char_span(start,end,label=label,alignment_mode="contract")
    if span is None:
      print("Skipping Entity")
    else:
      ents.append(span)
    filtered_ents=filter_spans(ents)
    doc.ents=filtered_ents
    doc_bin.add(doc)
doc_bin.to_disk("train.spacy")

100%|██████████| 31/31 [00:00<00:00, 112.53it/s]

Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity
Skipping Entity


In [ ]:
#before training we need a config file https://spacy.io/usage/training

In [32]:
!python -m spacy init fill-config base_config.cfg config.cfg
#modify max epochs (10) and batch size(100)

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-07-01 18:31:37,021] [INFO] Set up nlp object from config
[2022-07-01 18:31:37,041] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-07-01 18:31:37,047] [INFO] Created vocabulary
[2022-07-01 18:31:40,116] [INFO] Added vectors: en_core_web_lg
[2022-07-01 18:31:43,239] [INFO] Finished initializing nlp object
[2022-07-01 18:31:45,876] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    153.29    0.49    0.64    0.39    0.00
  7     200        501.32   3113.23   78.43   78.12   78.74    0.78
✔ Saved pipeline to output directory
model-last


In [75]:
nlp_ner=spacy.load("/content/model-best")

In [ ]:
doc = nlp_ner("While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.")

colors = {"PATHOGEN": "#F67DE3", "MEDICINE": "#7DF6D9", "MEDICALCONDITION":"#a6e22d"}
options = {"colors": colors}

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)